In [1]:
import numpy as np
import os
import shutil
import subprocess
import pandas as pd


In [2]:
df =pd.read_csv('node_info.csv')

node_info_dict = df.to_dict()

node_info_dict

{'node_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49},
 'region': {0: " 'europe-west3-c'",
  1: " 'europe-west3-c'",
  2: " 'europe-west3-c'",
  3: " 'europe-west3-c'",
  4: " 'europe-west3-c'",
  5: " 'europe-west3-c'",
  6: " 'europe-west3-c'",
  7: " 'europe-west3-c'",
  8: " 'europe-west3-c'",
  9: " 'europe-west3-c'",
  10: " 'europe-west3-c'",
  11: " 'europe-west3-c'",
  12: " 'europe-west3-c'",
  13: " 'europe-west3-c'",
  14: " 'europe-west3-c'",
  15: " 'europe-west3-c'",
  16: " 'europe-west3-c'",
  17: " 'europe-west3-c'",
  18: " 'europe-west3-

In [3]:
total_count = 48


replica_count = 24

n_clusters = int(total_count / replica_count)

nclients = n_clusters

In [4]:
# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b',\
#            'europe-west3-c', 'europe-west3-c', 'europe-west3-c', 'europe-west3-c', \
#            'asia-south1-c', 'asia-south1-c', 'asia-south1-c', 'asia-south1-c']


regions = [ 'europe-west3-c', 'asia-south1-c']




# regions = [ 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b']
# regions = [ 'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b']



# regions = ['europe-west3-c',  \
#            'asia-south1-c']


n_regions = len(regions)

In [5]:
for i,k in enumerate(regions):
    print(i,k)

0 europe-west3-c
1 asia-south1-c


In [6]:
lines  = []
zones = {}


lim = 0
for reg in regions:
    for i in range(replica_count+int(nclients/n_clusters)):
        line = 'gcloud compute instances create resdb-'+f"{lim+i:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
        
        print(line)
        lines.append(line)
        zones[i] = reg
    
    lim = lim + replica_count+int(nclients/n_clusters)
        
    
    
    
with open('launch_cloud_instances.sh', 'w') as f:
    for item in lines:
        f.write(str(item) + '\n')


gcloud compute instances create resdb-00 --project=ucr-ursa-major-lesani-lab --zone=europe-west3-c --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labe

In [7]:
import subprocess
import concurrent.futures

def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=len(commands)) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-01' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-02' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-03' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-05' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-12' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/z

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/asia-south1-c/instances/resdb-42' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/asia-south1-c/instances/resdb-40' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/asia-south1-c/instances/resdb-43' already exists



In [8]:
# os.system('sh launch_cloud_instances.sh')

In [9]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [10]:
# os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


instance_type = 'e2-small'


redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_internal_ips')



os.system('gcloud compute instances list --format="value(networkInterfaces[0].networkIP)" '+\
      redirect+' all_external_ips')












0

In [11]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [12]:
ipList, len(ipList)

(['10.156.0.95',
  '10.156.0.93',
  '10.156.0.7',
  '10.156.0.101',
  '10.156.0.94',
  '10.156.0.96',
  '10.156.0.115',
  '10.156.0.113',
  '10.156.0.103',
  '10.156.0.106',
  '10.156.0.110',
  '10.156.0.98',
  '10.156.0.109',
  '10.156.0.102',
  '10.156.0.104',
  '10.156.0.114',
  '10.156.0.99',
  '10.156.0.105',
  '10.156.0.112',
  '10.156.0.111',
  '10.156.0.107',
  '10.156.0.10',
  '10.156.0.100',
  '10.156.0.108',
  '10.156.0.97',
  '10.160.0.109',
  '10.160.0.92',
  '10.160.0.91',
  '10.160.0.105',
  '10.160.0.98',
  '10.160.0.88',
  '10.160.0.106',
  '10.160.0.96',
  '10.160.0.93',
  '10.160.0.2',
  '10.160.0.97',
  '10.160.0.94',
  '10.160.0.108',
  '10.160.0.18',
  '10.160.0.104',
  '10.160.0.95',
  '10.160.0.90',
  '10.160.0.89',
  '10.160.0.107',
  '10.160.0.100',
  '10.160.0.103',
  '10.160.0.11',
  '10.160.0.101',
  '10.160.0.102',
  '10.160.0.99'],
 50)

In [13]:
new_server_only_ips= []
new_client_only_ips= []



for i in range(total_count + nclients):
    if node_info_dict['ServerOrClientregion'][i] ==0:
        new_server_only_ips = new_server_only_ips + [ipList[i]]
    else:
        new_client_only_ips = new_client_only_ips + [ipList[i]]
        
        

In [14]:
# new_server_only_ips= []
# new_client_only_ips= []


# current_region_total = 0
# for i in range(n_regions):
#     print(current_region_total,current_region_total+ clusters_per_region_map[i] *replica_count)
#     new_server_only_ips = new_server_only_ips + ipList[current_region_total:current_region_total+ clusters_per_region_map[i] *replica_count]
    
#     new_client_only_ips = new_client_only_ips + ipList[current_region_total+ clusters_per_region_map[i] *replica_count:
#                                                       current_region_total+ clusters_per_region_map[i] *(replica_count+1)]
    
    
#     current_region_total = current_region_total + clusters_per_region_map[i] *replica_count +  clusters_per_region_map[i]
    
    

In [15]:
new_server_only_ips, len(new_server_only_ips)

(['10.156.0.95',
  '10.156.0.93',
  '10.156.0.7',
  '10.156.0.101',
  '10.156.0.94',
  '10.156.0.96',
  '10.156.0.115',
  '10.156.0.113',
  '10.156.0.103',
  '10.156.0.106',
  '10.156.0.110',
  '10.156.0.98',
  '10.156.0.109',
  '10.156.0.102',
  '10.156.0.104',
  '10.156.0.114',
  '10.156.0.99',
  '10.156.0.105',
  '10.156.0.112',
  '10.156.0.111',
  '10.156.0.107',
  '10.156.0.10',
  '10.156.0.100',
  '10.156.0.108',
  '10.160.0.109',
  '10.160.0.92',
  '10.160.0.91',
  '10.160.0.105',
  '10.160.0.98',
  '10.160.0.88',
  '10.160.0.106',
  '10.160.0.96',
  '10.160.0.93',
  '10.160.0.2',
  '10.160.0.97',
  '10.160.0.94',
  '10.160.0.108',
  '10.160.0.18',
  '10.160.0.104',
  '10.160.0.95',
  '10.160.0.90',
  '10.160.0.89',
  '10.160.0.107',
  '10.160.0.100',
  '10.160.0.103',
  '10.160.0.11',
  '10.160.0.101',
  '10.160.0.102'],
 48)

In [16]:
new_client_only_ips, len(new_client_only_ips) 

(['10.156.0.97', '10.160.0.99'], 2)

In [17]:
ipList = new_server_only_ips + new_client_only_ips

In [18]:
ipList, len(ipList)

(['10.156.0.95',
  '10.156.0.93',
  '10.156.0.7',
  '10.156.0.101',
  '10.156.0.94',
  '10.156.0.96',
  '10.156.0.115',
  '10.156.0.113',
  '10.156.0.103',
  '10.156.0.106',
  '10.156.0.110',
  '10.156.0.98',
  '10.156.0.109',
  '10.156.0.102',
  '10.156.0.104',
  '10.156.0.114',
  '10.156.0.99',
  '10.156.0.105',
  '10.156.0.112',
  '10.156.0.111',
  '10.156.0.107',
  '10.156.0.10',
  '10.156.0.100',
  '10.156.0.108',
  '10.160.0.109',
  '10.160.0.92',
  '10.160.0.91',
  '10.160.0.105',
  '10.160.0.98',
  '10.160.0.88',
  '10.160.0.106',
  '10.160.0.96',
  '10.160.0.93',
  '10.160.0.2',
  '10.160.0.97',
  '10.160.0.94',
  '10.160.0.108',
  '10.160.0.18',
  '10.160.0.104',
  '10.160.0.95',
  '10.160.0.90',
  '10.160.0.89',
  '10.160.0.107',
  '10.160.0.100',
  '10.160.0.103',
  '10.160.0.11',
  '10.160.0.101',
  '10.160.0.102',
  '10.156.0.97',
  '10.160.0.99'],
 50)

In [19]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]

In [20]:
clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(10000)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(20000)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(10000)
    clusterPortMap2[7001+i] = str(20000)

In [21]:
clusterPortMap1, clusterPortMap2

({0: '10000',
  1: '10000',
  2: '10000',
  3: '10000',
  4: '10000',
  5: '10000',
  6: '10000',
  7: '10000',
  8: '10000',
  9: '10000',
  10: '10000',
  11: '10000',
  12: '10000',
  13: '10000',
  14: '10000',
  15: '10000',
  16: '10000',
  17: '10000',
  18: '10000',
  19: '10000',
  20: '10000',
  21: '10000',
  22: '10000',
  23: '10000',
  24: '10000',
  25: '10000',
  26: '10000',
  27: '10000',
  28: '10000',
  29: '10000',
  30: '10000',
  31: '10000',
  32: '10000',
  33: '10000',
  34: '10000',
  35: '10000',
  36: '10000',
  37: '10000',
  38: '10000',
  39: '10000',
  40: '10000',
  41: '10000',
  42: '10000',
  43: '10000',
  44: '10000',
  45: '10000',
  46: '10000',
  47: '10000',
  7001: '10000',
  7002: '10000'},
 {0: '20000',
  1: '20000',
  2: '20000',
  3: '20000',
  4: '20000',
  5: '20000',
  6: '20000',
  7: '20000',
  8: '20000',
  9: '20000',
  10: '20000',
  11: '20000',
  12: '20000',
  13: '20000',
  14: '20000',
  15: '20000',
  16: '20000',
  17: '200

In [22]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 


> Task :compileJava

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar


Note: /home/tejas/work/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


> Task :installDist

BUILD SUCCESSFUL in 7s
3 actionable tasks: 3 executed


0

In [ ]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

In [ ]:
os.system('echo 3108 | sudo -S ./gradlew clean')

In [ ]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

In [ ]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [ ]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

In [ ]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [ ]:
replica_count

In [ ]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

In [ ]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

In [ ]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [ ]:
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

In [ ]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [ ]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any